<a href="https://colab.research.google.com/github/geraldm24/DS-Unit-2-Linear-Models/blob/master/Module3_LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [3]:
pd.set_option('display.max.rows', 110)
pd.set_option('display.max.columns',21)
print(df)

       BOROUGH             NEIGHBORHOOD            BUILDING_CLASS_CATEGORY  \
0            1                  CHELSEA    13 CONDOS - ELEVATOR APARTMENTS   
1            1                  FASHION                21 OFFICE BUILDINGS   
2            1                  FASHION                21 OFFICE BUILDINGS   
3            1   GREENWICH VILLAGE-WEST    13 CONDOS - ELEVATOR APARTMENTS   
4            1  UPPER EAST SIDE (59-79)  15 CONDOS - 2-10 UNIT RESIDENTIAL   
...        ...                      ...                                ...   
23035        4               ST. ALBANS            01 ONE FAMILY DWELLINGS   
23036        4                SUNNYSIDE       09 COOPS - WALKUP APARTMENTS   
23037        4                SUNNYSIDE     10 COOPS - ELEVATOR APARTMENTS   
23038        4                WOODHAVEN            02 TWO FAMILY DWELLINGS   
23039        4                 WOODSIDE      12 CONDOS - WALKUP APARTMENTS   

      TAX_CLASS_AT_PRESENT  BLOCK   LOT  EASE-MENT BUILDING_CLA

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [0]:
# SUBSET DATA TO THESE PARAMETERS
DF = df[(df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS')& (df['SALE_PRICE'] > 100000) & (df['SALE_PRICE']< 2000000)]

In [20]:
DF['SALE_DATE'] = pd.to_datetime(DF['SALE_DATE'], infer_datetime_format=True)
cutoff = pd.to_datetime('2019-04-01')

DF.SALE_DATE < cutoff

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


44        True
61        True
78        True
108       True
111       True
         ...  
23029    False
23031    False
23032    False
23033    False
23035    False
Name: SALE_DATE, Length: 3151, dtype: bool

In [0]:
# split the data so we can see how new data fits our model
train = DF[DF.SALE_DATE < cutoff]
test = DF[DF.SALE_DATE >= cutoff]

In [22]:
train

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,2019-01-01
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,2019-01-01
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000,2019-01-02
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000,2019-01-02
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000,2019-01-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18129,5,OTHER,01 ONE FAMILY DWELLINGS,1,4081,44,NaN,A2,10 SEAFOAM STREET,NaN,10306.0,1.0,0.0,1.0,"2,400",921.0,1950.0,1,A2,330000,2019-03-29
18130,5,OTHER,01 ONE FAMILY DWELLINGS,1,2373,201,NaN,A5,74 MCVEIGH AVE,NaN,10314.0,1.0,0.0,1.0,"2,450",2128.0,1980.0,1,A5,690000,2019-03-29
18132,5,OTHER,01 ONE FAMILY DWELLINGS,1,1132,42,NaN,A1,479 VILLA AVENUE,NaN,10302.0,1.0,0.0,1.0,"4,361",1807.0,2018.0,1,A1,610949,2019-03-29
18134,5,OTHER,01 ONE FAMILY DWELLINGS,1,3395,37,NaN,A2,63 NUGENT AVENUE,NaN,10305.0,1.0,0.0,1.0,"6,000",621.0,1930.0,1,A2,520000,2019-03-29


In [27]:
# before we use onehotencoder we must remove categorical columns with high amounts of elements(unique counts)
# those columns will increase the dimensionality and possibly take to long to run best case scenario
DF.describe(include='object').T.sort_values(by='unique')


,count,unique,top,freq
BUILDING_CLASS_CATEGORY,3151,1,01 ONE FAMILY DWELLINGS,3151
APARTMENT_NUMBER,1,1,RP.,1
TAX_CLASS_AT_PRESENT,3151,2,1,3111
BOROUGH,3151,5,4,1580
NEIGHBORHOOD,3151,7,OTHER,2959
BUILDING_CLASS_AT_TIME_OF_SALE,3151,11,A1,1186
BUILDING_CLASS_AT_PRESENT,3151,13,A1,1185
LAND_SQUARE_FEET,3151,1035,"4,000",289
ADDRESS,3151,3135,22-40 93RD STREET,2


In [28]:
DF.describe(include='object').columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_PRESENT', 'ADDRESS',
       'APARTMENT_NUMBER', 'LAND_SQUARE_FEET',
       'BUILDING_CLASS_AT_TIME_OF_SALE'],
      dtype='object')

In [30]:
DF.SALE_PRICE.mean()

628560.1126626468

In [34]:
DF.groupby('BOROUGH')['SALE_PRICE'].mean()

BOROUGH
1    971333.333333
2    485363.614334
3    767618.923650
4    648829.275316
5    539438.697832
Name: SALE_PRICE, dtype: float64

In [0]:
# now lets arrange our features 
target = 'SALE_PRICE'
drop_cats = ['NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY',
       'TAX_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_PRESENT', 'ADDRESS',
       'APARTMENT_NUMBER', 'LAND_SQUARE_FEET',
       'BUILDING_CLASS_AT_TIME_OF_SALE']
features = train.columns.drop([target]+ drop_cats)

In [0]:
# train the model and test the data
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [0]:
# encoding time the fit, fits the data to the model and transforms it to the new columns
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)

In [0]:
# now that we already did fit we will trust that and now just tranform the new columns to the test
X_test = encoder.transform(X_test)

In [45]:
X_test.describe().T

,count,mean,std,min,25%,50%,75%,max
BOROUGH_3,644.0,0.215839,0.411723,0.0,0.00,0.0,0.0,1.0
BOROUGH_4,644.0,0.583851,0.493302,0.0,0.00,1.0,1.0,1.0
BOROUGH_2,644.0,0.079193,0.270249,0.0,0.00,0.0,0.0,1.0
BOROUGH_5,644.0,0.118012,0.322874,0.0,0.00,0.0,0.0,1.0
BOROUGH_1,644.0,0.003106,0.055684,0.0,0.00,0.0,0.0,1.0
BLOCK,644.0,7539.559006,3854.438406,107.0,4500.00,7393.0,10857.5,16350.0
LOT,644.0,76.773292,174.379223,1.0,21.00,41.0,68.0,2202.0
EASE-MENT,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ZIP_CODE,644.0,11158.880435,410.161914,10030.0,11208.00,11362.0,11419.0,11697.0
RESIDENTIAL_UNITS,644.0,0.987578,0.124087,0.0,1.00,1.0,1.0,2.0


In [0]:
# a function to engineer_features
